In [1]:
import numpy
from collections import deque
import pandas
import math
import pandas_datareader.data as web
import datetime
import requests
import requests_cache
import xlrd
import tempfile
import itertools

/Users/justus/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def get_fred(fred_series):
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)
    
    start = datetime.datetime(1800, 1, 1)
    df = web.DataReader(fred_series, "fred", start, session=session)
    return df

# All FRED data can be found at https://fred.stlouisfed.org/series/SERIES_NAME
FRED_SERIES = [
    'INTGSBJPM193N', # 1966-2016 Interest Rates, Government Securities, Government Bonds for Japan
    'INTGSTJPM193N', # 1955-2016 Interest Rates, Government Securities, Treasury Bills for Japan
]

fred = get_fred(FRED_SERIES)
fred.head()

,INTGSBJPM193N,INTGSTJPM193N
DATE,,
1955-04-01,NaN,5.524
1955-05-01,NaN,5.524
1955-06-01,NaN,5.524
1955-07-01,NaN,5.524
1955-08-01,NaN,5.524


In [3]:
def iterate_fund(ladder, yield_curve, max_maturity):
    ladder.reduce_maturities()
    ladder.generate_payments()
    sold_bonds = ladder.sell_bonds(yield_curve)

    # Only buy a new bond if we actually sold one...
    if sold_bonds:
        ladder.buy_bond(yield_curve[max_maturity-1], max_maturity)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest does...)
    nav = ladder.get_nav(yield_curve)

    return (ladder, nav)

In [4]:
def a2m(annual_rate):
    return pow(annual_rate + 1, 1/12) - 1

class Bond:
    def __init__(self, face_value, yield_pct, maturity, payments_per_year=12):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        self.payments_per_year = payments_per_year
        
    def __repr__(self):
        return ('Maturity: %d | Yield: %.2f%% | Face Value: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))

    def gen_payment(self):
        return self.face_value * self.yield_pct / self.payments_per_year
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity / 12, (self.face_value * self.yield_pct), self.face_value)
        return -value
    
class BondLadder:
    def __init__(self, min_maturity, max_maturity):
        self.min_maturity = min_maturity
        self.max_maturity = max_maturity
        self.cash = 0
        
        self.ladder = set()
        
    def print_all(self):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond)
            
    def print_all_values(self, rates):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond.value(rates))
        
    def buy_bond(self, rate, maturity):
        b = Bond(self.cash, rate, maturity)
        self.add_bond(b)
        self.cash = 0
        return b
        
    def get_nav(self, rates):
        return self.cash + sum((b.value(rates) for b in self.ladder))

    def generate_payments(self):
        self.cash += sum((b.gen_payment() for b in self.ladder))        
        
    def __repr__(self):
        return ('%d-%d Ladder { Num Bonds: %d. }' % (self.max_maturity, self.min_maturity, len(self.ladder)))
        
    def add_bond(self, bond):
        #assert bond.maturity <= self.max_maturity
        #assert bond.maturity >= self.min_maturity
        self.ladder.add(bond)
    
    def reduce_maturities(self):
        for bond in self.ladder:
            bond.maturity -= 1

    def sell_bonds(self, rates):
        to_sell = filter(lambda bond: bond.maturity <= self.min_maturity, self.ladder)
        to_sell = list(to_sell)
        self.ladder = self.ladder.difference(to_sell)
        self.cash += sum((b.value(rates) for b in to_sell))
        return to_sell

In [5]:
def bootstrap(yield_curve, max_bonds, min_maturity):
    bond_yield = yield_curve[max_bonds - 1]

    # Why - 11?
    #min_maturity -= 11

    ladder = BondLadder(min_maturity, max_bonds)
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)

    for i, j in zip(range(max_bonds), range(min_maturity, max_bonds+1)):
        face_value = pow(1 + a2m(bond_yield), i) * starting_face_value
        b = Bond(face_value, bond_yield, j)
        ladder.add_bond(b)
    return ladder
bootstrap([.0532]*120, 10*12, 5*12)

120-60 Ladder { Num Bonds: 61. }

In [6]:
def splice_data(raw_rates, series):
    # Start by loading the data we get from Shiller.
    # This will always exist.

    def safe_add(series_index, rate_index):
        # Don't overwrite any data we already have.
        if math.isnan(series.iloc[series_index]):
            series.iloc[series_index] = raw_rates[rate_index]

    safe_add(1 * 12 - 1, 'INTGSTJPM193N')
    safe_add(10 * 12 - 1, 'INTGSBJPM193N')

def build_yield_curve(raw_rates, yield_curve_size=10*12):
    s = pandas.Series(math.nan, index=numpy.arange(yield_curve_size))

    # We use NaN to indicate "the data needs to be interpolated"
    # We have a few different data series that we splice together.
    splice_data(raw_rates, s)
    
    # This will do linear interpolation where it can.
    s.interpolate(inplace=True)
    
    # But it can still leave us with NaNs at the low end of the range
    s.fillna(method='backfill', inplace=True)
    
    # all of the data is in the form 3.71 but we want it to be .0371,
    # since that's what a percent actually is
    return s.apply(lambda x: x / 100).tolist()

In [7]:
['%.3f' % (s*100) for s in build_yield_curve(fred.iloc[15*12])]

['5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',
 '5.932',


In [8]:
bootstrap(build_yield_curve(fred.iloc[-2]), 10 * 12, 4 * 12)

120-48 Ladder { Num Bonds: 73. }

In [9]:
def loop(ladder, rates, max_maturity):
    df = pandas.DataFrame(columns=['NAV', 'Change'])

    # The first iterations have fake data with duplicate years
    # But that's okay because we overwrite them with later data
    # (since they all have the same year)
    for (year, current_rates) in rates:
        if year.year % 5 == 0 and year.month == 1:
            print('Calculating...', year.year)
        (ladder, nav) = iterate_fund(ladder, build_yield_curve(current_rates), max_maturity)
        df.loc[year] = {'NAV' : nav, 'Change' : None}

    calculate_returns(df)
    return df

def calculate_returns(df):
    # Longinvest calculates the return based on comparison's to
    # next year's NAV. So I'll do the same. Even though that seems
    # weird to me. Maybe it's because the rates are based on January?
    # Hmmm...that sounds plausible.
    max_row = df.shape[0]

    for i in range(max_row - 1):
        next_nav = df.iloc[i+1]['NAV']
        nav = df.iloc[i]['NAV']
        change = (next_nav - nav) / nav
        df.iloc[i]['Change'] = change
    return df

def make_annual_ladder(max_maturity, min_maturity, yields):
    rate = yields[max_maturity - 1]
    
    # We have to add the "- 12" in order to make things like up with how
    # longinvest runs things. His "10-4" ladder is really more of "10-3" ladder:
    # bonds get sold the moment they become a 3 year bond.
    ladder = BondLadder(min_maturity - 12, max_maturity)

    face_value = 50
    for i in range(min_maturity, max_maturity + 1, 12):
        ladder.add_bond(Bond(face_value, rate, i))
        face_value = face_value * (1 + rate)

    return ladder

def simulate_monthly_turnover(max_maturity, min_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = bootstrap(initial_yields, max_maturity, min_maturity)

    return loop(ladder, rates.iterrows(), max_maturity)

def simulate_annual_turnover(max_maturity, min_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = make_annual_ladder(max_maturity, min_maturity, initial_yields)

    # longinvest actually simulates 1870 and assumes 1871 rates. That's why,
    # when the simulation starts in January 1871, all the bonds have paid 1 year
    # of interest and one of the bonds is ready to be sold.
    # So we need to generate 11 months of fake data to do the same simulation.
    # Why 11 months? The 12th month is the real January 1871 data.
    first_index = rates.head(1).index
    bootstrap_rates = itertools.repeat(next(rates.iterrows()), 11)
    all_rates = itertools.chain(bootstrap_rates, rates.iterrows())

    return loop(ladder, all_rates, max_maturity)

In [10]:
%%time
sim_results = simulate_monthly_turnover(10, 4, fred)
print(sim_results.head())

/Users/justus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: numpy.pv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pv instead (https://pypi.org/project/numpy-financial/).


Calculating... 1960
Calculating... 1965
Calculating... 1970
Calculating... 1975
Calculating... 1980
Calculating... 1985
Calculating... 1990
Calculating... 1995
Calculating... 2000
Calculating... 2005
Calculating... 2010
Calculating... 2015
                    NAV    Change
1955-04-01  4327.879291  0.004603
1955-05-01  4347.801962  0.004603
1955-06-01  4367.816344  0.004603
1955-07-01  4387.922858  0.004603
1955-08-01  4408.121930  0.004603
CPU times: user 8.44 s, sys: 149 ms, total: 8.59 s
Wall time: 9.89 s


In [11]:
sim_results.to_csv('bonds-monthly-japan-fred.csv')